In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)
library(dplyr)
library(DESeq2)
library(plotly)
library(ggplot2)
library(viridis)
library(magrittr)
library(pheatmap)
library(DescTools)
library(pdfCluster)
library(RColorBrewer)
library(SummarizedExperiment)
library(caret)
library(class)
library(htmlwidgets)
set.seed(88888888) # maximum luck

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/"

GTEx_DATA_DIR <- IN_DIR
GTEx_DATA_FIL <- "rse_tcdd_data.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(GTEx_DATA_DIR,GTEx_DATA_FIL,sep=""))

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [ ]:
%%R

variances <- apply(assay(final_result), 1, var)

zero_variance_rows <- which(variances == 0)

if (length(zero_variance_rows) > 0) {
    deleted_data <- assay(final_result)[zero_variance_rows, , drop = FALSE]
    save(deleted_data, file=paste0(OUT_DIR, "deleted_data_90.RData"))
    new_assays <- assay(final_result)[-zero_variance_rows, , drop = FALSE]
    new_row_data <- rowData(final_result)[-zero_variance_rows, , drop = FALSE]

    new_final_result <- SummarizedExperiment(
        assays = SimpleList(counts = new_assays),
        rowData = new_row_data,
        colData = colData(final_result)
    )
    
    final_result <- new_final_result
    
    print(dim(assay(final_result)))
    print(dim(rowData(final_result)))
} else {
    cat("No rows with zero variance found.\n")
}

print(head(assay(final_result)))

[1] 43252   383
[1] 43252    11
                     [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10] [,11]
ENSMUSG00000079800.2    0    0    0    0    0    0    0    0   49    49     0
ENSMUSG00000095092.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079192.2    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000079794.2    0    0    0    0    0    0    0    0    0     0    50
ENSMUSG00000094799.1    0    0    0    0    0    0    0    0    0     0     0
ENSMUSG00000095250.1    0    0    0    0    0   50    0    0    0     0     0
                     [,12] [,13] [,14] [,15] [,16] [,17] [,18] [,19] [,20]
ENSMUSG00000079800.2    50     0    50    50     0     0     0     0     0
ENSMUSG00000095092.1     0     0     0     0     0     0     0     0     0
ENSMUSG00000079192.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000079794.2     0     0     0     0     0     0     0     0     0
ENSMUSG00000094799.1     0     0     0     0   

ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0     50     48      0      0      0
                     [,142] [,143] [,144] [,145] [,146] [,147] [,148] [,149]
ENSMUSG00000079800.2      0      0      0      0      0      0      0      0
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,150] [,151] [,152] [,153] [,154] [,155] [,156] [,157]

ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,262] [,263] [,264] [,265] [,266] [,267] [,268] [,269]
ENSMUSG00000079800.2      0      0      0      0      0    100    150     50
ENSMUSG00000095092.1      0      0      0      0      0      0      0      0
ENSMUSG00000079192.2      0      0      0      0      0      0      0      0
ENSMUSG00000079794.2      0      0      0      0      0      0      0      0
ENSMUSG00000094799.1      0      0      0      0      0      0      0      0
ENSMUSG00000095250.1      0      0      0      0      0      0      0      0
                     [,270] [,271] [,272] [,273] [,274] [,275] [,276] [,277]
ENSMUSG00000079800.2      0      0     50      0     50     40     34     49

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
%%R
library(tibble)

keep_samples <- colData(final_result)$dose %in% c(0.00, 30.00)

final_result <- final_result[, keep_samples]

temp_df <- as_tibble(colData(final_result)) %>%
  filter(!grepl("SRP131784", study))

tcdd_data <- temp_df
final_result <- final_result[, colData(final_result)$external_id %in% temp_df$external_id]

print(dim(final_result))
print(dim(tcdd_data))


[1] 43252   136
[1] 136 198


In [ ]:
%%R
sampling_percentage <- 0.90

tcdd_data_dose0 <- filter(tcdd_data, dose == 0)
tcdd_data_dose30 <- filter(tcdd_data, dose == 30)

final_results_list <- list()

for (i in 1:10) {
  min_count <- min(nrow(tcdd_data_dose0), nrow(tcdd_data_dose30)) * sampling_percentage
  min_count <- floor(min_count)  

  sampled_tcdd_dose0 <- sample_n(tcdd_data_dose0, size = min_count)
  sampled_tcdd_dose30 <- sample_n(tcdd_data_dose30, size = min_count)

  tcdd_data <- bind_rows(sampled_tcdd_dose0, sampled_tcdd_dose30)

  temp_final_result <- final_result[, colData(final_result)$external_id %in% tcdd_data$external_id]
  matched_indices <- match(tcdd_data$external_id, colData(temp_final_result)$external_id)
  temp_final_result <- temp_final_result[, matched_indices]

  final_results_list[[i]] <- temp_final_result

  print(paste("Iteration", i, "dimensions:"))
  print(dim(tcdd_data))
  print(dim(temp_final_result))
}


[1] "Iteration 1 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 2 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 3 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 4 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 5 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 6 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 7 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 8 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 9 dimensions:"
[1] 122 198
[1] 43252   122
[1] "Iteration 10 dimensions:"
[1] 122 198
[1] 43252   122


In [7]:
%%R
gtex_df <- read.csv("/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/gtex/input/node_features2.txt",sep="")

In [ ]:
%%R
index = 0
for(final_result in final_results_list){
    print(paste("-----------------------round=", index, "---------------------",sep=""))
    tcdd_data <- colData(final_result)
    dose_counts <- table(tcdd_data$dose)
    dose_counts_df <- as.data.frame(dose_counts)
    colnames(dose_counts_df) <- c("Dose", "Count")
    combined_gender <- ifelse(tcdd_data$Sex != "", tcdd_data$Sex, tcdd_data$gender)
    write.table(tcdd_data$project_id,file=paste(OUT_DIR,"tcdd_project_id030_time_cross_90", index, ".txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
    write.table(combined_gender,file=paste(OUT_DIR,"tcdd_gender030_time_cross_90",index,".txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
    write.table(tcdd_data$external_id,file=paste(OUT_DIR,"tcdd_sample_id030_time_cross_90",index,".txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
    write.table(tcdd_data$dose,file=paste(OUT_DIR,"tcdd_dose030_time_cross_90",index,".txt",sep=""), row.names = FALSE, col.names = FALSE, sep = "\t")
    tcdd.df <- final_result %>% SummarizedExperiment::assay() %>% as.data.frame()
    colnames(tcdd.df) <- colData(final_result)$external_id
    ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(tcdd.df))
    deleted_ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(deleted_data))
    rownames(tcdd.df) <- ensembl_wo_ids
    rownames(deleted_data) <- deleted_ensembl_wo_ids
    reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
    tcdd.df <- tcdd.df[reactome_ensembl_ids,]

    tcdd.dose.detail.vec <- tcdd_data$dose
    tcdd_dose_detail.vec <- tcdd_data$dose
    scale.factor <- (.Machine$integer.max - 1) / max(tcdd.df)
    tcdd.df <- round(tcdd.df * scale.factor)
    tcdd.df <- tcdd.df + 1
    dds <- DESeq2::DESeqDataSetFromMatrix(countData = as.matrix(tcdd.df),
                                      colData = data.frame(Sample=colnames(tcdd.df),
                                                           Dose=tcdd.dose.detail.vec),
                                      design = ~ Dose)
    vst.counts <- DESeq2::vst(dds,
                          blind = FALSE,
                          fitType = "local")
    ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")
    rxn2ensembls.nls <- list()
    rxns_w_tcdd_ensembls.df <- ensembl2rxns.df %>% dplyr::filter(V1 %in% reactome_ensembl_ids)
    rxns_w_tcdd_ensembls.df$V1 <- as.character(rxns_w_tcdd_ensembls.df$V1)
    rxns_w_tcdd_ensembls.df$V2 <- as.character(rxns_w_tcdd_ensembls.df$V2)


    for(i in 1:nrow(rxns_w_tcdd_ensembls.df)){
      ens_id <- rxns_w_tcdd_ensembls.df$V1[i]
      rxn_id <- rxns_w_tcdd_ensembls.df$V2[i]
    #     print(names(rxn2ensembls.nls))

      ensembl_list_for_rxn_id <- rxn2ensembls.nls[[rxn_id]]
      if(is.null(ensembl_list_for_rxn_id)){
    #     print("null")
        ensembl_list_for_rxn_id <- c(ens_id)
      }
      rxn2ensembls.nls[[rxn_id]] <- c(ensembl_list_for_rxn_id,ens_id) %>% unique()
    }
    
    N_FOLDS <- 10
    sample <- colData(vst.counts)
    vst.count.mtx <-
    vst.counts %>% SummarizedExperiment::assay() %>% as.data.frame()
    rxns <- rxn2ensembls.nls %>% names()
    rxn_knn_misclass_rate.nls <- list()
    rxn_knn_ari.nls <- list()
    rxn_knn_ecount.nls <- list()
    rxn_pca.nls <- list()
    count <- 0
    toi_indices <- seq(1,length(tcdd_dose_detail.vec))
    tcdd_dose_detail_vec_tis_of_interest <-
    tcdd_dose_detail.vec[toi_indices]
    vst.count.mtx.tis_of_interest <- vst.count.mtx[, toi_indices]
    training_indices <-
       caret::createDataPartition(
          tcdd_dose_detail_vec_tis_of_interest,
          times = 1,
          p = 1.0, # no data will be held out when set to "1.0"
          list = FALSE
       )
    vst.count.mtx.train <-
    vst.count.mtx.tis_of_interest[, training_indices] #9/10ths of data
    vst.count.mtx.test  <-
    vst.count.mtx.tis_of_interest[, -training_indices] #1/10th of data
    tcdd_dose_detail.vec.train <- tcdd_dose_detail_vec_tis_of_interest[training_indices]
    
    Y <- tcdd_dose_detail.vec.train
    X <- readRDS(paste(OUT_DIR, "rxn_pca_nls030_time_cross_90",index,".Rds", sep = ""))
    E <- read.table(paste(IN_DIR,"ReactionNetwork_Rel.txt",sep=""))
    rxn2nodeLabel.nls <- list()
    nodeLabel2rxn.nls <- list()
    for(i in 1:length(X)){
      rxn2nodeLabel.nls[[names(X)[i]]] <- i
      nodeLabel2rxn.nls[[i]] <- names(X)[i]
    }
    E <- E %>%
    dplyr::filter(V1 %in% names(rxn2nodeLabel.nls))
    E <- E %>%
    dplyr::filter(V3 %in% names(rxn2nodeLabel.nls))
    E <- E %>%
    dplyr::filter(V1 %in% names(rxn2nodeLabel.nls)) %>%
    dplyr::filter(V3 %in% names(rxn2nodeLabel.nls)) %>%
    dplyr::select(V1,V3)
    node1 <- numeric()
    node2 <- numeric()
    for(i in 1:nrow(E)){
      node1 <- c(node1,rxn2nodeLabel.nls[[as.character(E$V1[i])]])
      node2 <- c(node2,rxn2nodeLabel.nls[[as.character(E$V3[i])]])
    }
    z <- unlist(rxn2nodeLabel.nls)
    y <- unlist(nodeLabel2rxn.nls)
    E <- data.frame(node1 = node1,
                node2 = node2)
    X <- as.data.frame(X)
    Y <- as.data.frame(Y)
    missing_in_gtex <- setdiff(names(gtex_df), names(X))
    missing_pathway <- data.frame(missing_pathway = missing_in_gtex)
    all_values <- as.vector(as.matrix(X))
    median_value <- median(all_values, na.rm = TRUE)
    for (col in missing_pathway$missing_pathway) {
      X[[col]] <- median_value
    }

    write.table(X,
          file=paste(IN_DIR,"node_features030_2_time_cross_90", index, ".txt",sep=""),
          row.names = FALSE,
          col.names = FALSE)
    write.table(Y,
              file=paste(IN_DIR,"graph_targets030_time_cross_90", index, ".txt",sep=""),
              row.names = FALSE,
              col.names = FALSE)
    index <- index + 1
}
print("finished")

[1] "-----------------------round=0---------------------"


In [ ]:
import os.path as op
import random
import time

import matplotlib.pyplot as plt
import numpy
import sklearn
import torch
import torch.nn.functional as nn_func
from sklearn import preprocessing
from sklearn.metrics import adjusted_rand_score
from torch.nn import Linear
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GraphConv, global_mean_pool

INPUT_CHANNELS = 1
OUTPUT_CHANNELS = 26
NEW_CHANNELS = 2
HIDDEN_CHANNELS = 64
BATCH_SIZE = 64
BENCHMARKING = False
EPOCHS = 500

random.seed = 88888888

In [ ]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()

        self.conv1 = GraphConv(INPUT_CHANNELS, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, OUTPUT_CHANNELS)

    def forward(self, x, edge_index, batch, edge_weight=None):
        # 1. Obtain node embeddings
        x = self.conv1(x, edge_index, edge_weight)
        x = x.relu()
        x = self.conv2(x, edge_index, edge_weight)
        x = x.relu()
        x = self.conv3(x, edge_index, edge_weight)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        #x = nn_func.dropout(x, training=self.training)
        x = self.lin(x)

        return x


def read_reactome_graph(e_fn):
    e_v1 = []
    e_v2 = []

    for line in open(e_fn, 'r'):
        dt = line.split()
        node1 = int(dt[0]) - 1  # subtracting to convert R idx to python idx
        node2 = int(dt[1]) - 1  # " "
        e_v1.append(node1)
        e_v2.append(node2)

    return e_v1, e_v2


def build_reactome_graph_datalist(e_v1, e_v2, n_fn, g_fn, pid_fn, sid_fn, gen_fn, dose_fn):
    edge_index = torch.tensor([e_v1, e_v2], dtype=torch.long)
    feature_v = numpy.loadtxt(n_fn)
    target_v = numpy.loadtxt(g_fn, dtype=float, delimiter=",")
    projectID_v = numpy.loadtxt(pid_fn, dtype=str, delimiter="\t")
    sampleID_v = numpy.loadtxt(sid_fn, dtype=str, delimiter="\t")
    gender_v = numpy.loadtxt(gen_fn, dtype=str, delimiter="\t")
    dose_v = numpy.loadtxt(dose_fn, dtype=str, delimiter="\t")
    
    binary_labels = (target_v > 0).astype(int)
    
    print("labels check:")
    for dose, label in zip(target_v[:10], binary_labels[:10]): 
        print(f"dose: {dose}, label: {label}")


    d_list = []
    for row_idx in range(len(feature_v)):
        features = feature_v[row_idx, :]
        x = torch.tensor(features, dtype=torch.float)
        x = x.unsqueeze(1)
#         y = torch.tensor([target_v[row_idx]])
        y = torch.tensor([binary_labels[row_idx]], dtype=torch.long)
        
        pid = projectID_v[row_idx]
        sid = sampleID_v[row_idx]
        gen = gender_v[row_idx]
        dose = dose_v[row_idx]
        
        d_list.append(Data(x=x, y=y, pid=pid, sid=sid, gen=gen, dose=dose, edge_index=edge_index))

    return d_list


def build_reactome_graph_loader(d_list, batch_size):
    loader = DataLoader(d_list, batch_size=batch_size, shuffle=False)  # True)

    return loader


def train(loader, dv):
    model.train()

    correct = 0
    for batch in loader:  # Iterate in batches over the training dataset.
        batch.validate()
        x = batch.x.to(dv)
        e = batch.edge_index.to(dv)
        b = batch.batch.to(dv)
        y = batch.y.to(dv)

        out = model(x, e, b)  # Perform a single forward pass.
        loss = criterion(out, y)  # Compute the loss.
        loss.backward()  # Derive gradients.
        optimizer.step()  # Update parameters based on gradients.
        optimizer.zero_grad()  # Clear gradients.
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == y).sum())  # Check against ground-truth labels.
    return correct / len(loader.dataset)  # Derive ratio of correct predictions.


def test(loader, dv):
    model.eval()

    targets = []
    predictions = []
    project_ids = []
    sample_ids = []
    genders = []
    doses = []
    confidences = []
    for batch in loader:  # Iterate in batches over the test dataset.
        x = batch.x.to(dv)
        e = batch.edge_index.to(dv)
        b = batch.batch.to(dv)
        y = batch.y.to(dv)
        targets += torch.Tensor.tolist(y)
        
        project_ids += batch.pid
        sample_ids += batch.sid
        genders += batch.gen
        doses += batch.dose
        
        out = model(x, e, b)  # Perform a single forward pass.
        prob = torch.softmax(out, dim=1)
        
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        predictions += torch.Tensor.tolist(pred)
        confidences += torch.Tensor.tolist(prob)
        
    num_classes = len(confidences[0])

    data_to_save = []
    for i in range(len(targets)):
        row = [project_ids[i], sample_ids[i], genders[i], doses[i], targets[i], predictions[i]] + confidences[i]
        data_to_save.append(row)
    data_to_save = numpy.array(data_to_save)
    print(data_to_save)
    
    fmt = ['%s', '%s', '%s', '%s', '%s', '%s'] + ['%s' for _ in range(num_classes)]
    
    headers = ['project_ids', 'sample_ids', 'genders', 'doses', 'target', 'prediction'] + [f'confidence_class_{i}' for i in range(num_classes)]
    numpy.savetxt(output_fn, data_to_save, fmt='\t'.join(fmt), delimiter='\t', header='\t'.join(headers), comments='')
        
    ari = adjusted_rand_score(targets, predictions)
    print(f'ari: {ari}')
    return ari

In [ ]:
for i in range(10):
    node_features_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/input/node_features030_2_time_cross_90{i}.txt'
    graph_targets_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/input/graph_targets030_time_cross_90{i}.txt'
    edges_fn = '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/gtex/input/edges.txt'
    model_fn = '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/gtex/GNN/trained_pytorch_model_rewired10_fold_full_dataset.pt'
    output_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/predictions030_time_cross_90{i}.tsv'
    sample_id_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/tcdd_sample_id030_time_cross_90{i}.txt'
    project_id_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/tcdd_project_id030_time_cross_90{i}.txt'
    gender_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/tcdd_gender030_time_cross_90{i}.txt'
    dose_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/tcdd_dose030_time_cross_90{i}.txt'
    # test graph_targets.txt, node_features.txt and edges.txt
    features_exist = op.exists(node_features_fn)
    targets_exist = op.exists(graph_targets_fn)
    edges_exist = op.exists(edges_fn)
    model_exists = op.exists(model_fn)

    def change_key(self, old, new):
        for _ in range(len(self)):
            k, v = self.popitem(False)
            self[new if old == k else k] = v

    (edge_v1, edge_v2) = read_reactome_graph(edges_fn)
    model = GNN(hidden_channels=HIDDEN_CHANNELS)
    device = cpu = torch.device('cpu')

    sd = torch.load(model_fn, map_location=device)
    change_key(sd, 'conv1.lin_l.weight', 'conv1.lin_rel.weight')
    change_key(sd, 'conv1.lin_l.bias', 'conv1.lin_rel.bias')
    change_key(sd, 'conv1.lin_r.weight', 'conv1.lin_root.weight')
    change_key(sd, 'conv2.lin_l.weight', 'conv2.lin_rel.weight')
    change_key(sd, 'conv2.lin_l.bias', 'conv2.lin_rel.bias')
    change_key(sd, 'conv2.lin_r.weight', 'conv2.lin_root.weight')
    change_key(sd, 'conv3.lin_l.weight', 'conv3.lin_rel.weight')
    change_key(sd, 'conv3.lin_l.bias', 'conv3.lin_rel.bias')
    change_key(sd, 'conv3.lin_r.weight', 'conv3.lin_root.weight')
    change_key(sd, 'lin.weight', 'lin.weight')
    change_key(sd, 'lin.bias', 'lin.bias')

    model.load_state_dict(sd)
    model.eval()
    
    # replace final layer with new shape matching new dataset
    model.lin = Linear(HIDDEN_CHANNELS, NEW_CHANNELS)

    model.conv1.lin_rel.weight.requires_grad = False
    model.conv1.lin_rel.bias.requires_grad = False
    model.conv1.lin_root.weight.requires_grad = False
    model.conv2.lin_rel.weight.requires_grad = False
    model.conv2.lin_rel.bias.requires_grad = False
    model.conv2.lin_root.weight.requires_grad = False
    model.conv3.lin_rel.weight.requires_grad = False
    model.conv3.lin_rel.bias.requires_grad = False
    model.conv3.lin_root.weight.requires_grad = False
    model.lin.weight.requires_grad = True
    model.lin.bias.requires_grad = True

    # for name, param in model.named_parameters(): print(name, param)

    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()))
    criterion = torch.nn.CrossEntropyLoss()
    
    data_list = build_reactome_graph_datalist(edge_v1, edge_v2, node_features_fn, graph_targets_fn, project_id_fn, sample_id_fn, gender_fn, dose_fn)
    # retrain model for fine tuning transfer learning
    train_data_list = data_list  # all data
    print(len(train_data_list))
    print(f'Number of training graphs: {len(train_data_list)}')
    train_data_loader = build_reactome_graph_loader(train_data_list, BATCH_SIZE)
    for epoch in range(EPOCHS):
        train(train_data_loader, device)
        train_acc = train(train_data_loader, device)
        print(f'Epoch: {epoch}, Train Acc: {train_acc}')
        if train_acc == 1.0:
            break

    final_ari = test(train_data_loader, device)
    print(f'test_ari: {final_ari}')

    model_save_name = f'tuned_pytorch_tcdd_model030_time_cross_90{i}.pt'
    path = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/{model_save_name}'
    torch.save(model.state_dict(), path)
    print(f'model saved as {path}')

In [ ]:
import os.path as op
import time
import torch
import numpy
import random
import sklearn
import numpy as np
import networkx as nx
import torch.nn.functional as F
from torch.nn import Linear
from sklearn import preprocessing
from collections import defaultdict
from IPython.display import Javascript
from torch_geometric.utils import to_networkx
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data, DataLoader
from captum.attr import Saliency, IntegratedGradients
from torch_geometric.nn import GraphConv, global_mean_pool

In [ ]:
# magic numbers
INPUT_CHANNELS = 1
OUTPUT_CHANNELS = 2
HIDDEN_CHANNELS = 64

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()

        self.conv1 = GraphConv(INPUT_CHANNELS, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, OUTPUT_CHANNELS)

    def forward(self, x, edge_index, batch, edge_weight=None):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index, edge_weight)
        x = x.relu()
        x = self.conv2(x, edge_index, edge_weight)
        x = x.relu()
        x = self.conv3(x, edge_index, edge_weight)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, training=self.training)
        x = self.lin(x)

        return x


def read_reactome_graph(e_fn):
    e_v1 = []
    e_v2 = []

    for line in open(e_fn, 'r'):
        dt = line.split()
        node1 = int(dt[0]) - 1  # subtracting to convert R idx to python idx
        node2 = int(dt[1]) - 1  # " "
        e_v1.append(node1)
        e_v2.append(node2)

    return e_v1, e_v2


def build_reactome_graph_datalist(e_v1, e_v2, n_features_fn, g_targets_fn):
    edge_index = torch.tensor([e_v1, e_v2], dtype=torch.long)
    feature_v = numpy.loadtxt(n_features_fn)
    t_v = numpy.loadtxt(g_targets_fn, dtype=float, delimiter=",")

#     t_encoder = sklearn.preprocessing.LabelEncoder()
#     t_v = t_encoder.fit_transform(t_v)

    binary_labels = (t_v > 0).astype(int)

    d_list = []
    for row_idx in range(len(feature_v)):
        x = torch.tensor(feature_v[row_idx, :], dtype=torch.float)
        x = x.unsqueeze(1)
#         y = torch.tensor([t_v[row_idx]])
        y = torch.tensor([binary_labels[row_idx]], dtype=torch.long)
        d_list.append(Data(x=x, y=y, edge_index=edge_index))

    return d_list


def explain(m, dt, target):
    input_mask = torch.ones(dt.edge_index.shape[1]).requires_grad_(True).to(device)
    if m == 'ig':
        ig = IntegratedGradients(model_forward)
        mask = ig.attribute(input_mask, target=target,
                            additional_forward_args=(dt,),
                            internal_batch_size=dt.edge_index.shape[1])
    else:
        raise Exception('Unknown explanation method')

    e_mask = np.abs(mask.cpu().detach().numpy())
    if e_mask.max() > 0:  # avoid division by zero
        e_mask = e_mask / e_mask.max()
    return e_mask


def aggregate_edge_directions(e_mask, dt):
    edge_mask_dict = defaultdict(float)
    for val, u, v in list(zip(e_mask, *dt.edge_index)):
        u, v = u.item(), v.item()
        if u > v:
            u, v = v, u
        edge_mask_dict[(u, v)] += val
    return edge_mask_dict


def model_forward(e_mask, dt):
    batch = torch.zeros(dt.x.shape[0], dtype=int).to(device)
    out = model(dt.x,
                dt.edge_index,
                batch,
                e_mask)
    return out

In [ ]:
for i in range(10):
    node_features_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/input/node_features030_2_time_cross_90",{i},".txt'
    graph_targets_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/input/graph_targets030_time_cross_90",{i},".txt'
    edges_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/gtex/input/edges.txt'
    model_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/tuned_pytorch_tcdd_model030_time_cross_90",{i},".pt'
    output_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/predictions030_time_cross_90",{i},".tsv'
    transformed_targets_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/transformed_targets030_time_cross_90",{i},".txt'
    inverted_targets_fn = f'/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/inverted_targets030_time_cross_90",{i},".txt'
    
    (edge_v1, edge_v2) = read_reactome_graph(edges_fn)

    data_list = build_reactome_graph_datalist(edge_v1, edge_v2, node_features_fn, graph_targets_fn)
    data_loader = DataLoader(data_list)

    # rebuild label encoder to invert numerical transformation
    target_v = numpy.loadtxt(graph_targets_fn, dtype=str, delimiter=",")
    target_encoder = sklearn.preprocessing.LabelEncoder()

    target_v = target_encoder.fit_transform(target_v)
    path = transformed_targets_fn
    numpy.savetxt(path, target_v, delimiter=",", fmt="%.0f")
    print(F"target_v saved as {path}")

    target_l = target_encoder.inverse_transform(target_v)
    path = inverted_targets_fn
    numpy.savetxt(path, target_l, delimiter=",", fmt="%s")
    print(F"target_l saved as {path}")

    model = GNN(hidden_channels=HIDDEN_CHANNELS)
    device = cpu = torch.device('cpu')
    model = model.to(device)
    path = model_fn
    model.load_state_dict(torch.load(path, map_location=device))
    model.eval()

    d = data_loader.dataset[0]
    d.edge_index.shape[1]

    data = data_loader.dataset[0]

    for target_tissue in range(2):
        title = 'Integrated Gradients'
        method = 'ig'
        data.to(device)
        print(F"processing tissue {target_tissue} with {title}, a.k.a. {method}")
        edge_mask = explain(method, data, target=target_tissue)
        # edge_mask_dict = aggregate_edge_directions(edge_mask, data)
        path = F"/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/learning_curve_analysis/output/{method}_{target_tissue}030_time_cross_90",{i},".txt"
        numpy.savetxt(path, edge_mask, delimiter=",")
        print(F"{method} {target_tissue} edges saved as {path}")
